In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

# Set random seed for purposes of reproducibility
seed = 21

from keras.datasets import cifar10

# loading in the data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
model.add(Activation('relu'))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(class_num))
model.add(Activation('softmax'))

epochs = 1
optimizer = 'adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#print(model.summary())

#numpy.random.seed(seed)
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0         
                                                        

In [ ]:
import tensorflow as tf
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
def grad_by_one_sample(model, x, y):
  with tf.GradientTape() as tape:
    pred = model(x[None, :])
    loss_by_one_sample = (y[None, :] - pred)**2
    grads = tape.gradient(loss_by_one_sample, model.trainable_variables)
  return grads

In [ ]:
grads_list = []

In [ ]:
def perform_one_epoch(model, opt, X_train, y_train, X_test, y_test, batch_size):

  for i_start, i_stop in zip(range(0, len(X_train) - batch_size, batch_size),
                             range(batch_size, len(X_train), batch_size)):
    
    def lambda_grad(var):
      return grad_by_one_sample(model, *var)
    l_grads = list(map(lambda_grad,zip(X_train[i_start: i_stop],
                                           y_train[i_start: i_stop])))
    grads = [tf.zeros_like(g) for g in l_grads[0]]
    for i in range(len(grads)):
      for j in range(len(l_grads)):
        grads[i] += l_grads[j][i]
    for i in range(len(grads)):
      grads[i] = grads[i] / len(l_grads)

    grads_list.append(grads)
    opt.apply_gradients(zip(grads, model.trainable_variables))

  test_pred = model(X_test)
  test_loss = tf.reduce_mean((y_test - test_pred)**2)
  return test_loss

In [ ]:
def train_model(model,
                opt,
                X_train,
                y_train,
                X_test,
                y_test,
                batch_size,
                epoch_num = 10):
  for epoch in range(epoch_num):
    test_loss = perform_one_epoch(model, opt, X_train, y_train, X_test, y_test, batch_size)
    print(f'epoch: {epoch}, test_loss: {test_loss}')

In [ ]:
train_model(model,
            opt,
            X_train,
            y_train,
            X_test,
            y_test,
            batch_size=16,
            epoch_num=10)

In [ ]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))